# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [69]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')

In [70]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_KEY")

# Import Data

In [71]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [127]:
# Define Panel Visualization Functions
sfo_data_by_year = sfo_data.groupby("year")
sfo_data_by_year_neighborhood = sfo_data.groupby(["year", "neighborhood"])
sfo_data_by_neighborhood = sfo_data.groupby(["neighborhood"])

# Create average sales price dataframe as a global variable
sales_price_sqr_foot = round(sfo_data_by_year_neighborhood[["sale_price_sqr_foot", "housing_units", "gross_rent"]].mean(),2).reset_index()

# Creata a data frame of the top 10 most expensive that can be used globally 
avg_sale_price = sfo_data_by_neighborhood[["sale_price_sqr_foot", "housing_units", "gross_rent"]].mean()
top_10_sale_price = avg_sale_price.sort_values(by="sale_price_sqr_foot", ascending=False).reset_index().head(10)
top_10_sale_price = round(top_10_sale_price, 2)


def housing_units_per_year():
    """Housing Units Per Year."""
    
    mean_housing = sfo_data_by_year["housing_units"].mean()
    housing_units = mean_housing.hvplot(
        kind="bar", 
        title="Housing Units in San Francisco 2010-2016", 
        ylim=(370000,387500),
        ylabel="Housing Units",
        xlabel="Year")
    
    return housing_units


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    average_gross = sfo_data_by_year[["gross_rent"]].mean().hvplot(
        title="Average Gross Rent in San Francisco",
        ylabel="Gross Rent",
        xlabel="Year")
    
    return average_gross


def average_sales_price():
    """Average Sales Price Per Year."""
    
    average_sale = sfo_data_by_year[["sale_price_sqr_foot"]].mean().hvplot(
        title="Average Sale Price per Square Foot in San Francisco",
        ylabel="Avg Sale Price",
        xlabel="Year")
    
    return average_sale


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    sales_price_plot = sales_price_sqr_foot[["year", "neighborhood", "sale_price_sqr_foot"]].hvplot(
        'year', 
        groupby="neighborhood",
        xlabel="Year",
        ylabel="Neighborhood")
    
    return sales_price_plot


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    most_expensive = top_10_sale_price[["neighborhood", "sale_price_sqr_foot"]].hvplot.bar(x="neighborhood", xlabel="Neighborhood", rot=90, ylabel="Avg Sale Price per Square Foot")
    
    return most_expensive  


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    top_10_coordinates_plot = px.parallel_coordinates(top_10_sale_price, color="sale_price_sqr_foot")

    return top_10_coordinates_plot


def parallel_categories():
    """Parallel Categories Plot."""
    
    top_10_categories_plot = px.parallel_categories(top_10_sale_price, color="sale_price_sqr_foot")
    
    return top_10_categories_plot


def neighborhood_map():
    """Neighborhood Map"""
    
    neighborhood_values_locations = pd.concat([avg_sale_price.reset_index(), df_neighborhood_locations.rename(columns=({"Neighborhood":"neighborhood", "Lat":"lat", "Lon":"lon"}))], axis=1)

    px.set_mapbox_access_token(mapbox_token)
    
    neighborhood_plot = px.scatter_mapbox(
    neighborhood_values_locations,
    lat="lat",
    lon="lon",
    size="sale_price_sqr_foot",
    color="gross_rent",
    color_continuous_scale=px.colors.cyclical.IceFire,
    title="Average Sale Price Per Square Foot and Gross Rent in San Francisco",
    zoom=10,
    width=500)
    
    return neighborhood_plot


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [128]:
welcome_column = pn.Column(
    "This Dashboard gives you the ability to view the historical prices of housing units, sales price per square foot, and gross rent across different neighborhoods in San Francisco.",
    neighborhood_map
)

yearly_market_column = pn.Column(
    housing_units_per_year, average_gross_rent, average_sales_price
)

neighborhood_column = pn.Column(
    average_price_by_neighborhood, top_most_expensive_neighborhoods
)

parallel_plot_columns = pn.Column(
    parallel_coordinates, parallel_categories
)

panel = pn.Tabs(
    ("Welcome", welcome_column),
    ("Yearly Market Analysis", yearly_market_column),
    ("Neighborhood Analysis", neighborhood_column),
    ("Parallel Plots Analysis", parallel_plot_columns)
)

## Serve the Panel Dashboard

In [129]:
panel.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive10823')
    [1] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive10830')
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive10912')
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive10994')
    [2] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive11077')
                    [0] HoloViews(DynamicMap, name='interactive11077')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive11113')
    [3] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive11197')
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive11203')